In [43]:
!pip install python-dotenv
!pip install requests
import os
import requests
from dotenv import load_dotenv
load_dotenv()  # Carrega o .env do diretório atual


True

In [44]:
import zipfile
import os

# Caminhos
zip_file_path = "/content/202401_NFs.zip"  # Arquivo ZIP
extraction_path = "/content/dados_extraidos/"  # Pasta de destino

# Criar pasta se não existir
os.makedirs(extraction_path, exist_ok=True)

# Extrair
with zipfile.ZipFile(zip_file_path, 'r') as zip_ref:
    zip_ref.extractall(extraction_path)
    print(f"Arquivos extraídos em: {extraction_path}")

# Verificar conteúdo
!ls -lh {extraction_path}

Arquivos extraídos em: /content/dados_extraidos/
total 336K
-rw-r--r-- 1 root root  43K Jun 15 18:18 202401_NFs_Cabecalho.csv
-rw-r--r-- 1 root root 292K Jun 15 18:18 202401_NFs_Itens.csv


In [45]:
load_dotenv()
deepseek_api_key = os.getenv("DEEPSEEK_API_KEY")
print("API Key:", deepseek_api_key)

API Key: sk-b73aa97c57fe4aa8b35e705af765c864


In [ ]:
!pip install streamlit pyngrok langchain_experimental langchain-openai langchain-community pandas

from langchain_community.chat_models import ChatOpenAI
import streamlit as st
from langchain_experimental.agents import create_csv_agent
import pandas as pd
import glob
from pyngrok import ngrok


DEEPSEEK_API_KEY = deepseek_api_key


llm = ChatOpenAI(
    openai_api_key=DEEPSEEK_API_KEY,
    openai_api_base="https://api.deepseek.com/v1",
    model_name="deepseek-chat"
)



# Carregar dados CSV
diretorio = glob.glob("dados_extraidos" + "/*.csv")
df_csv = pd.concat([pd.read_csv(arquivo) for arquivo in diretorio], ignore_index=True)

st.write("Dados carregados:", df_csv.head())

question = st.text_input("Qual a sua pergunta sobre as notas fiscais?")

if question:
    agent = create_csv_agent(llm, df_csv, verbose=True)
    response = agent.run(question)
    st.write("Resposta:", response)



In [47]:
!curl -v http://localhost:8080

*   Trying 127.0.0.1:8080...
* Connected to localhost (127.0.0.1) port 8080 (#0)
> GET / HTTP/1.1
> Host: localhost:8080
> User-Agent: curl/7.81.0
> Accept: */*
> 
* Mark bundle as not supporting multiuse
< HTTP/1.1 404 Not Found
< Date: Sun, 15 Jun 2025 18:18:19 GMT
< Connection: keep-alive
< Content-Length: 0
< 
* Connection #0 to host localhost left intact
